In [1]:
import pandas as pd
import numpy as np

## GRPM calculation
- GRPMld
- GRPMvd

In [2]:
df1= pd.read_excel('/content/DFORE-199_ITR2_3_4_Variance_V4.0.xlsx', sheet_name='ITR-2')
df2= pd.read_excel('/content/DFORE-199_ITR2_3_4_Variance_V4.0.xlsx', sheet_name='ITR-3')
df3= pd.read_excel('/content/DFORE-199_ITR2_3_4_Variance_V4.0.xlsx', sheet_name='ITR-4')

In [4]:
df2.columns

Index(['as400id', 'cohort_flag', 'vendor', 'branchname', 'accountname',
       'revenue', 'no_of_transacted_machines', 'installcap',
       'first_machine_vend_Date', 'no_of_day_vends', 'no_of_all_days',
       'GRPM_AllDays', 'GRPM_transacted_days'],
      dtype='object')

In [3]:
df3.isnull().sum()

as400id                      0
cohort_flag                  0
vendor                       0
branchname                   0
accountname                  0
revenue                      0
no_of_transacted_machines    0
installcap                   0
first_machine_vend_Date      0
no_of_day_vends              0
no_of_all_days               0
GRPM_AllDays                 0
GRPM_transacted_days         0
dtype: int64

In [6]:
def calc_grpm(x):
  x['GRPMld']= 30.42*x['revenue']/(max(x['no_of_all_days'],1)*max(x['installcap'],1))
  x['GRPMvd']= 30.42*x['revenue']/(max(x['no_of_day_vends'],1)*max(x['installcap'],1))
  return(x)

In [7]:
df1= df1.apply(calc_grpm, axis=1)
df2= df2.apply(calc_grpm, axis=1)
df3= df3.apply(calc_grpm, axis=1)

In [12]:
df1['iteration']= 2
df2['iteration']= 3
df3['iteration']= 4
df= pd.concat([df1,df2,df3], axis=0)

## Machines

### Transacted Machines verswus Installed Cap

In [16]:
mf= df.pivot(index=['cohort_flag', 'as400id'], columns=['iteration'], values=['no_of_transacted_machines','installcap' ]).fillna(0)

In [21]:
for c in [2,3,4]:
  cln= 'used_pct'+ str(c)
  mf[cln]= mf['no_of_transacted_machines'][c]/mf['installcap'][c]
  

In [27]:
(mf['used_pct2']> 1)	or (mf['used_pct3']> 1)

ValueError: ignored

## Checking 2 flavors of GRPM between DB and Python

In [ ]:
print(sum(df1.GRPMld.between(df1.GRPM_AllDays- 0.0001, df1.GRPM_AllDays+ 0.0001)))
print(sum(df1.GRPMvd.between(df1.GRPM_transacted_days- 0.0001, df1.GRPM_transacted_days+ 0.0001)))

4000
4000


In [ ]:
print(sum(df2.GRPMld.between(df2.GRPM_AllDays- 0.0001, df2.GRPM_AllDays+ 0.0001)))
print(sum(df2.GRPMvd.between(df2.GRPM_transacted_days- 0.0001, df2.GRPM_transacted_days+ 0.0001)))

4394
4394


## GRPM Description

In [ ]:
df.groupby(['iteration', 'cohort_flag'])['GRPMld', 'GRPMvd'].agg('describe').T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


iteration              2                        3             
cohort_flag      control         test     control         test
GRPMld count  754.000000  3246.000000  834.000000  3560.000000
       mean    43.011603    45.957954   42.686655    46.208999
       std     36.392258    55.162950   36.273445    88.794043
       min      0.020040     0.016078    0.015379     0.018281
       25%     14.338282    17.160534   14.336217    16.653264
       50%     37.836315    38.924859   35.005466    38.451486
       75%     61.576374    63.290809   60.554651    63.696359
       max    285.187500  2368.957500  243.747292  4862.854286
GRPMvd count  754.000000  3246.000000  834.000000  3560.000000
       mean    54.455930    56.812328   54.712945    57.853690
       std     37.087464    48.707635   38.641173    83.875308
       min      0.330652     0.543214    0.330652     0.543214
       25%     28.023654    29.986993   26.988753    29.256616
       50%     47.357788    50.236867   47.035114    50.739464
       75%     73.082170    74.368845   72.885880    75.082794
       max    285.187500  1895.166000  380.250000  4538.664000

In [ ]:
df.loc[df.as400id==df.loc[df.GRPMld > 4862, 'as400id'].values[0]]

,as400id,cohort_flag,vendor,branchname,accountname,revenue,no_of_transacted_machines,installcap,first_machine_vend_Date,no_of_day_vends,no_of_all_days,GRPM_AllDays,GRPM_transacted_days,GRPMld,GRPMvd,iteration
2271,220676,test,CSCGo,Balt/Wash,OAKFIELD APARTMENT HOMES,311.5,44,1,2021-09-13,5,4,2368.957500,1895.166,2368.957500,1895.166,2
2331,220676,test,CSCGo,Balt/Wash,OAKFIELD APARTMENT HOMES,2238.0,90,1,2021-09-13,15,14,4862.854286,4538.664,4862.854286,4538.664,3


In [ ]:
as400list= df.loc[df.installcap < df.no_of_transacted_machines,'as400id'].unique().tolist()

### Unique machines ransacted > installed Cap

In [ ]:
df.loc[df.as400id.isin(as400list)].groupby(['iteration', 'cohort_flag'])['GRPMld', 'GRPMvd'].agg('describe').T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


iteration              2                        3             
cohort_flag      control         test     control         test
GRPMld count   10.000000    42.000000   11.000000    55.000000
       mean    48.869923   123.722866   59.321328   165.572763
       std     47.263451   359.469663   49.049670   647.926327
       min      2.614219     2.589600   10.828859     2.681927
       25%     13.661190    36.186837   16.249681    38.336845
       50%     39.685077    54.646534   58.580024    62.980667
       75%     67.089496    83.791832   80.188351   106.579397
       max    157.630909  2368.957500  166.585714  4862.854286
GRPMvd count   10.000000    42.000000   11.000000    55.000000
       mean    67.176723   114.096416   78.566220   166.676784
       std     35.887801   286.734407   42.637108   603.529622
       min     23.527969    14.259375   34.602750    15.970500
       25%     41.685740    37.353492   50.347227    47.564741
       50%     64.170068    58.381955   66.058324    70.749545
       75%     79.715187    87.482963   93.812284   111.922892
       max    144.495000  1895.166000  159.013636  4538.664000

### Other Machines

In [ ]:
df.loc[~df.as400id.isin(as400list)].groupby(['iteration', 'cohort_flag'])['GRPMld', 'GRPMvd'].agg('describe').T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


iteration              2                        3             
cohort_flag      control         test     control         test
GRPMld count  744.000000  3204.000000  823.000000  3505.000000
       mean    42.932862    44.938564   42.464321    44.335959
       std     36.258720    36.721312   36.060358    36.212427
       min      0.020040     0.016078    0.015379     0.018281
       25%     14.393879    17.077562   14.337009    16.457381
       50%     37.722528    38.716364   34.946786    38.190479
       75%     61.450344    62.919492   59.798839    63.180000
       max    285.187500   308.002500  243.747292   397.089643
GRPMvd count  744.000000  3204.000000  823.000000  3505.000000
       mean    54.284952    56.061413   54.394129    56.146052
       std     37.096961    36.159867   38.513501    36.650795
       min      0.330652     0.543214    0.330652     0.543214
       25%     27.921441    29.939521   26.617500    29.127150
       50%     47.108093    50.193000   46.894208    50.481466
       75%     73.013223    74.165140   72.760226    74.782500
       max    285.187500   388.767600  380.250000   388.767600

### Iteration 2 to 3 GRPM Changes

In [ ]:
gf=df.pivot(index=['cohort_flag', 'as400id'], columns=['iteration'], values=['GRPMld'])

In [ ]:
gf['GRPMIter2_3']=(gf['GRPMld'][3]- gf['GRPMld'][2])/gf['GRPMld'][2]

In [ ]:
gf.dropna(inplace=True)

In [ ]:
gf.loc[~gf.GRPMIter2_3.between(-.20,.20)]

GRPMld            GRPMIter2_3
iteration                    2          3            
cohort_flag as400id                                  
control     16215     1.652277   4.881094    1.954162
            30487    21.216222  26.723125    0.259561
            32134    86.731568  45.430821   -0.476190
            32135    15.843750  24.173036    0.525714
            32184    53.537472  65.628333    0.225839
...                        ...        ...         ...
test        7208182   4.095000   2.314565   -0.434783
            7208338   4.436250   5.630625    0.269231
            7400582  15.935932  20.589833    0.292038
            7404284  23.544773  28.810476    0.223646
            7404397  11.407500   7.333393   -0.357143

[637 rows x 3 columns]

In [ ]:
gf['itr2dec']=pd.qcut(gf['GRPMld'][2], 10, labels=[1,2,3,4,5,6,7,8,9,10], )

In [ ]:
gf['itr3dec']=pd.qcut(gf['GRPMld'][3], 10, labels=[1,2,3,4,5,6,7,8,9,10], )

In [ ]:
gf.itr3dec= gf.itr3dec.astype('int')

In [ ]:
gf.itr2dec= gf.itr2dec.astype('int')

In [ ]:
gf['PrcntlChng']= abs(gf.itr3dec- gf.itr2dec)

In [ ]:
gf.sort_values('PrcntlChng', ascending=False)

GRPMld              ... itr3dec PrcntlChng
iteration                     2           3  ...                   
cohort_flag as400id                          ...                   
test        1420987   24.716250  115.161429  ...      10          6
            6823799  112.385000   32.955000  ...       5          5
control     6823814   59.889375   17.745000  ...       3          5
test        4426385   73.515000   27.522857  ...       4          5
control     63384     63.797500   25.279583  ...       4          4
...                         ...         ...  ...     ...        ...
test        708485     0.508668    0.402695  ...       1          0
            708498    29.659500   31.941000  ...       5          0
            708505    39.825426   42.660764  ...       6          0
            708599    35.008140   32.506894  ...       5          0
control     16141     77.204577   76.923167  ...       9          0

[4000 rows x 6 columns]

In [ ]:
gf.reset_index().to_csv('GRPMiter2iter3.csv')

In [ ]:
gf.loc[gf.PrcntlChng > 2].reset_index().to_csv('GRPMiter2iter3.csv')

In [ ]:
gas400lst=gf.loc[gf.PrcntlChng > 2].reset_index().loc[:,'as400id'].tolist()

In [ ]:
len(gas400lst)

31

## Base Satistics Calculation

In [ ]:
sf1= df1.groupby('cohort_flag')['GRPMld', 'GRPMvd'].agg([np.size, np.mean, np.std, np.var])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [ ]:
sf2= df2.groupby('cohort_flag')['GRPMld', 'GRPMvd'].agg([np.size, np.mean, np.std, np.var])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [ ]:
sf2

GRPMld                        ...     GRPMvd                        
               size       mean        std  ...       mean        std          var
cohort_flag                                ...                                   
control       834.0  42.686655  36.273445  ...  54.712945  38.641173  1493.140221
test         3560.0  46.208999  88.794043  ...  57.853690  83.875308  7035.067298

[2 rows x 8 columns]